In [1]:
# importing libraries


import requests
import pandas as pd
import time

In [2]:
# Stating API key


API_KEY = 'This is Definitly the API KEY'
CHANNEL_ID = 'UC0xAECQagqOaucQw-6UZPAg'

In [3]:
# Make API call

pageToken = ''


url = "https://www.googleapis.com/youtube/v3/search?key="+API_KEY+"&channelId="+CHANNEL_ID+"&part=snippet,id&order=date&maxResults=10000&"+pageToken
response = requests.get(url).json()

response['items'][0]

{'kind': 'youtube#searchResult',
 'etag': 'T1ki9_alB3csvTjB1UgwdTramlQ',
 'id': {'kind': 'youtube#video', 'videoId': 'gvjjSjbeb1c'},
 'snippet': {'publishedAt': '2021-11-12T23:00:36Z',
  'channelId': 'UC0xAECQagqOaucQw-6UZPAg',
  'title': 'How to Night Crab in Oregon - Netted 30+ Dungies',
  'description': 'Night crabbing in Tillamook. We netted 30+ dungies over two nights. It was very dark and a lot of fun. Remember to bring a good ...',
  'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/gvjjSjbeb1c/default.jpg',
    'width': 120,
    'height': 90},
   'medium': {'url': 'https://i.ytimg.com/vi/gvjjSjbeb1c/mqdefault.jpg',
    'width': 320,
    'height': 180},
   'high': {'url': 'https://i.ytimg.com/vi/gvjjSjbeb1c/hqdefault.jpg',
    'width': 480,
    'height': 360}},
  'channelTitle': 'MT Adventures',
  'liveBroadcastContent': 'none',
  'publishTime': '2021-11-12T23:00:36Z'}}

In [4]:
# Grabbing video ID


video_id = response['items'][0]["id"]['videoId']

video_id

'gvjjSjbeb1c'

In [5]:
# Grabbing video title


video_title = response['items'][0]['snippet']["title"]
video_title = str(video_title).replace('&amp;','')

video_title

'How to Night Crab in Oregon - Netted 30+ Dungies'

In [6]:
# grabbing upload date


upload_date = response['items'][0]["snippet"]["publishTime"]
upload_date = str(upload_date).split("T")[0]

upload_date

'2021-11-12'

In [7]:
def get_video_details(video_id):

    #collecting view, like, dislike, comment counts
    url_video_stats = "https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&part=statistics&key="+API_KEY
    response_video_stats = requests.get(url_video_stats).json()

    view_count = response_video_stats['items'][0]['statistics']['viewCount']
    like_count = response_video_stats['items'][0]['statistics']['likeCount']
    
    # comment_count = response_video_stats['items'][0]['statistics']['commentCount'] 
    # this account doesn't have comments turned on
    
    return view_count, like_count    #, comment_count

In [8]:
# making a function to get video info


def get_videos(df):
    # Make API call
    pageToken = ''

    url = "https://www.googleapis.com/youtube/v3/search?key="+API_KEY+"&channelId="+CHANNEL_ID+"&part=snippet,id&order=date&maxResults=10000&"+pageToken
    response = requests.get(url).json()
    time.sleep(1)
    
    for video in response['items']:
        if video['id']['kind'] == 'youtube#video':
            video_id = video["id"]['videoId']
            video_title = video['snippet']["title"]
            video_title = str(video_title).replace('&amp;','')
            upload_date = video["snippet"]["publishTime"]
            upload_date = str(upload_date).split("T")[0]

            view_count, like_count = get_video_details(video_id)

            # save data to dataframe
            df = df.append({
                'video_id': video_id,
                'video_title': video_title,
                'upload_date': upload_date,
                'view_count': view_count,
                'like_count': like_count,
                # 'comment_count': comment_count         this account does NOT have comments turned on
            }, ignore_index = True)
        
    return df

In [9]:
# building dataframe


df = pd.DataFrame(columns=["video_id",'video_title','upload_date','view_count','like_count'])

df = get_videos(df)

df

,video_id,video_title,upload_date,view_count,like_count
0,gvjjSjbeb1c,How to Night Crab in Oregon - Netted 30+ Dungies,2021-11-12,923,23
1,UxlPGYiOXho,Netarts Bay Crabbing - Almost Limit Out,2021-11-11,1390,18
2,NdNqAwelLJI,Crabbing along the Oregon Coast - 100+ crabs,2021-11-11,1800,28
3,cIjmEpdGZrw,Crabbing for Lunch at Nehalem Bay - $17.50 lb,2021-08-01,767,20
4,sx425fsBU4Y,Harvesting Mussels on the Oregon Coast - Netar...,2021-07-31,387,8
5,VycTbbXMFxg,Oregon Coast Gaper Clamming at Netarts Bay - D...,2021-07-31,227,5
6,83jAELDjdWA,Haleakala Downhill Biking in Maui - 6500&#39;,2021-06-21,144,6
7,2XA8dRe7DAQ,Harvesting Mussels and Gaper Clams at Netarts Bay,2021-06-03,474,11
8,xj54enWsbwo,87 Gapers at Netarts Bay in an Hour,2021-06-02,715,14
9,yduuo5NnJL8,Snorkeling Kapalua Bay in Maui - Spotted Half ...,2021-04-10,250,12


In [10]:
df.to_csv('MTAyoutubeapi.csv', index= False)